#**멀티미디어 컴퓨팅**<Br>

<br>
1. 코드를 모두 실행시킨다
2. flask code 부분에 있는 코드를 실행시키면 나오는 ngrok 링크(.io)에 접속한다. 
3. 이후 

# 라이브러리

In [ ]:
import os
import gdown
import shutil 
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import subprocess

In [ ]:
!pip install flask_ngrok
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template

In [ ]:
os.mkdir('templates')
os.mkdir('static')
%cd static/
os.mkdir('anime_outputs')
os.mkdir('seg_outputs')
os.mkdir('splicing_outputs')
os.mkdir('inputs')
%cd /content/

/content/static
/content


# Github 불러오기

In [ ]:
!git clone https://github.com/TachibanaYoshino/AnimeGANv2

Cloning into 'AnimeGANv2'...
remote: Enumerating objects: 1679, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 1679 (delta 0), reused 0 (delta 0), pack-reused 1676
Receiving objects: 100% (1679/1679), 996.82 MiB | 36.88 MiB/s, done.
Resolving deltas: 100% (110/110), done.


In [ ]:
!pip install tensorflow==1.15
!git clone https://github.com/bryandlee/animegan2-pytorch.git
%cd animegan2-pytorch
!mkdir inputs
!mkdir outputs
%cd ..

     |████████████████████████████████| 412.3MB 35kB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 3.8MB 41.6MB/s 
     |████████████████████████████████| 512kB 58.1MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7557 sha256=7040f1405c6eeb5aaa275687bd554fb42d9517e271f067780c1dacc6682ed583
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.12.1 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
ERROR: kapre 0.3.5 has requirement tensorflow>=2.0.0, but you'll have tensorflow 1.15.0 which is incompatible.
  Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensor

In [ ]:
!pip install ninja
!git clone https://github.com/PeikeLi/Self-Correction-Human-Parsing
%cd Self-Correction-Human-Parsing
!mkdir checkpoints
!mkdir inputs
!mkdir outputs
%cd ..

     |████████████████████████████████| 112kB 14.9MB/s 
Cloning into 'Self-Correction-Human-Parsing'...
remote: Enumerating objects: 719, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 719 (delta 3), reused 5 (delta 0), pack-reused 708
Receiving objects: 100% (719/719), 3.80 MiB | 19.43 MiB/s, done.
Resolving deltas: 100% (145/145), done.
/content/Self-Correction-Human-Parsing
/content


In [ ]:
!git clone https://github.com/menyifang/ADGAN.git

Cloning into 'ADGAN'...
remote: Enumerating objects: 167, done.
remote: Counting objects: 100% (167/167), done.
remote: Compressing objects: 100% (141/141), done.
remote: Total 167 (delta 33), reused 153 (delta 22), pack-reused 0
Receiving objects: 100% (167/167), 18.67 MiB | 6.30 MiB/s, done.
Resolving deltas: 100% (33/33), done.


# Style Transfer

In [ ]:
%cd /content
%cd animegan2-pytorch

/content
/content/animegan2-pytorch


In [ ]:
text = '''
#!/usr/bin/env python
# -*- encoding: utf-8 -*-

import sys
sys.path.append('/content')

import argparse
import numpy as np
import os

import tensorflow as tf
from AnimeGANv2.net import generator as tf_generator

import torch
from model import Generator


def load_tf_weights(tf_path):
    test_real = tf.placeholder(tf.float32, [1, None, None, 3], name='test')
    with tf.variable_scope("generator", reuse=False):
        test_generated = tf_generator.G_net(test_real).fake

    saver = tf.train.Saver()

    with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, device_count = {'GPU': 0})) as sess:
        ckpt = tf.train.get_checkpoint_state(tf_path)

        assert ckpt is not None and ckpt.model_checkpoint_path is not None, f"Failed to load checkpoint {tf_path}"

        saver.restore(sess, ckpt.model_checkpoint_path)
        print(f"Tensorflow model checkpoint {ckpt.model_checkpoint_path} loaded")

        tf_weights = {}
        for v in tf.trainable_variables():
            tf_weights[v.name] = v.eval()
    
    return tf_weights

            
def convert_keys(k):

    # 1. divide tf weight name in three parts [block_idx, layer_idx, weight/bias]
    # 2. handle each part & merge into a pytorch model keys
    
    k = k.replace("Conv/", "Conv_0/").replace("LayerNorm/", "LayerNorm_0/")    
    keys = k.split("/")[2:]
    
    is_dconv = False

    # handle C block..
    if keys[0] == "C":
        if keys[1] in ["Conv_1", "LayerNorm_1"]:
            keys[1] = keys[1].replace("1", "5")
        
        if len(keys) == 4:
            assert "r" in keys[1]

            if keys[1] == keys[2]:
                is_dconv = True
                keys[2] = "1.1"
            
            block_c_maps = {
                "1":  "1.2",
                "Conv_1":  "2",
                "2":  "3",
            }
            if keys[2] in block_c_maps:
                keys[2] = block_c_maps[keys[2]]

            keys[1] = keys[1].replace("r", "") + ".layers." + keys[2]
            keys[2] = keys[3]
            keys.pop(-1)
    assert len(keys) == 3

    # handle output block
    if "out" in keys[0]:
        keys[1] = "0"
    
    # first part
    if keys[0] in ["A", "B", "C", "D", "E"]:
        keys[0] = "block_" + keys[0].lower()        
        
    # second part
    if "LayerNorm_" in keys[1]:
        keys[1] = keys[1].replace("LayerNorm_", "") + ".2"
    if "Conv_" in keys[1]:
        keys[1] = keys[1].replace("Conv_", "") + ".1"
        
    # third part
    keys[2] = {
        "weights:0": "weight",
        "w:0": "weight",
        "bias:0": "bias",
        "gamma:0": "weight",
        "beta:0": "bias",
    }[keys[2]]
        
    return ".".join(keys), is_dconv


def convert_and_save(tf_checkpoint_path, save_name):

    tf_weights = load_tf_weights(tf_checkpoint_path)
    
    torch_net = Generator()
    torch_weights = torch_net.state_dict()

    torch_converted_weights = {}
    for k, v in tf_weights.items():
        torch_k, is_dconv = convert_keys(k)
        assert torch_k in torch_weights, f"weight name mismatch: {k}"

        converted_weight = torch.from_numpy(v)
        if len(converted_weight.shape) == 4:
            if is_dconv:
                converted_weight = converted_weight.permute(2, 3, 0, 1)
            else:
                converted_weight = converted_weight.permute(3, 2, 0, 1)

        assert torch_weights[torch_k].shape == converted_weight.shape, f"shape mismatch: {k}"

        torch_converted_weights[torch_k] = converted_weight

    assert sorted(list(torch_converted_weights)) == sorted(list(torch_weights)), f"some weights are missing"
    torch_net.load_state_dict(torch_converted_weights)    
    torch.save(torch_net.state_dict(), save_name)
    print(f"PyTorch model saved at {save_name}")
    

if __name__ == '__main__':

    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--tf_checkpoint_path',
        type=str,
        default='AnimeGANv2/checkpoint/generator_Paprika_weight',
    )
    parser.add_argument(
        '--save_name', 
        type=str, 
        default='pytorch_generator_Paprika.pt',
    )
    args = parser.parse_args()
    
    convert_and_save(args.tf_checkpoint_path, args.save_name)
'''
file = open("convert_weights2.py","w")
file.write(text)
file.close()

In [ ]:
# test code 만들기
text = '''
import sys
sys.path.append('/content/animegan2-pytorch')

import torch
import cv2
import numpy as np
import os

from model import Generator
import easydict

torch.backends.cudnn.enabled = False
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True


def load_image(image_path, x32=False):
    img = cv2.imread(image_path).astype(np.float32)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]

    if x32:  # resize image to multiple of 32s
        def to_32s(x):
            return 256 if x < 256 else x - x % 32

        img = cv2.resize(img, (to_32s(w), to_32s(h)))

    img = torch.from_numpy(img)
    img = img / 127.5 - 1.0
    return img


def test(args):
    device = args.device

    net = Generator()
    net.load_state_dict(torch.load(args.checkpoint, map_location="cpu"))
    net.to(device).eval()
    print(f"model loaded: {args.checkpoint}")

    os.makedirs(args.output_dir, exist_ok=True)

    for image_name in sorted(os.listdir(args.input_dir)):
        if os.path.splitext(image_name)[-1].lower() not in [".jpg", ".png", ".bmp", ".tiff"]:
            continue

        image = load_image(os.path.join(args.input_dir, image_name), args.x32)

        with torch.no_grad():
            input = image.permute(2, 0, 1).unsqueeze(0).to(device)
            out = net(input, args.upsample_align).squeeze(0).permute(1, 2, 0).cpu().numpy()
            out = (out + 1) * 127.5
            out = np.clip(out, 0, 255).astype(np.uint8)

        cv2.imwrite(os.path.join(args.output_dir, image_name), cv2.cvtColor(out, cv2.COLOR_BGR2RGB))
        print(f"image saved: {image_name}")


if __name__ == '__main__':

    args = easydict.EasyDict({

        "checkpoint": '/content/pytorch_generator_Hayao.pt',

        "input_dir": '/content/static/inputs',

        "output_dir": '/content/static/anime_outputs',

        "device": 'cpu',

        "upsample_align": False,

        "x32": "store_true"

    })


    test(args)

'''
file = open("anime_execute.py","w")
file.write(text)
file.close()

In [ ]:
%cd /content
! python /content/animegan2-pytorch/convert_weights2.py --tf_checkpoint_path '/content/AnimeGANv2/checkpoint/generator_Hayao_weight' --save_name 'pytorch_generator_Hayao.pt'

/content


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `layer.__call__` method instead.





2021-06-08 06:41:28.406735: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2021-06-08 06:41:28.411601: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000175000 Hz
2021-06-08 06:41:28.411854: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55e70b4ff480 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-06-08 06:41:28.411885: I tensorflow/compiler/xla/servi

# paprika version

In [ ]:
%cd /content
%cd animegan2-pytorch

/content
/content/animegan2-pytorch


In [ ]:
# test code 만들기
text = '''
import sys
sys.path.append('/content/animegan2-pytorch')

import torch
import cv2
import numpy as np
import os

from model import Generator
import easydict

torch.backends.cudnn.enabled = False
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True


def load_image(image_path, x32=False):
    img = cv2.imread(image_path).astype(np.float32)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]

    if x32:  # resize image to multiple of 32s
        def to_32s(x):
            return 256 if x < 256 else x - x % 32

        img = cv2.resize(img, (to_32s(w), to_32s(h)))

    img = torch.from_numpy(img)
    img = img / 127.5 - 1.0
    return img


def test(args):
    device = args.device

    net = Generator()
    net.load_state_dict(torch.load(args.checkpoint, map_location="cpu"))
    net.to(device).eval()
    print(f"model loaded: {args.checkpoint}")

    os.makedirs(args.output_dir, exist_ok=True)

    for image_name in sorted(os.listdir(args.input_dir)):
        if os.path.splitext(image_name)[-1].lower() not in [".jpg", ".png", ".bmp", ".tiff"]:
            continue

        image = load_image(os.path.join(args.input_dir, image_name), args.x32)

        with torch.no_grad():
            input = image.permute(2, 0, 1).unsqueeze(0).to(device)
            out = net(input, args.upsample_align).squeeze(0).permute(1, 2, 0).cpu().numpy()
            out = (out + 1) * 127.5
            out = np.clip(out, 0, 255).astype(np.uint8)

        cv2.imwrite(os.path.join(args.output_dir, image_name), cv2.cvtColor(out, cv2.COLOR_BGR2RGB))
        print(f"image saved: {image_name}")


if __name__ == '__main__':

    args = easydict.EasyDict({

        "checkpoint": '/content/pytorch_generator_Paprika.pt',

        "input_dir": '/content/static/inputs',

        "output_dir": '/content/static/anime_outputs',

        "device": 'cpu',

        "upsample_align": False,

        "x32": "store_true"

    })


    test(args)

'''
file = open("anime_execute2.py","w")
file.write(text)
file.close()

In [ ]:
%cd /content
! python /content/animegan2-pytorch/convert_weights2.py 

/content


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `layer.__call__` method instead.





2021-06-08 06:41:34.113208: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2021-06-08 06:41:34.118208: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2000175000 Hz
2021-06-08 06:41:34.118484: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55d107a79480 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-06-08 06:41:34.118513: I tensorflow/compiler/xla/servi

#인물사진 segmentation

In [ ]:
%cd /content
%cd Self-Correction-Human-Parsing

/content
/content/Self-Correction-Human-Parsing


In [ ]:
dataset = 'atr'         #select from ['lip', 'atr', 'pascal']

if dataset == 'lip':
    url = 'https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH'
elif dataset == 'atr':
    url = 'https://drive.google.com/uc?id=1ruJg4lqR_jgQPj-9K0PP-L2vJERYOxLP'
elif dataset == 'pascal':
    url = 'https://drive.google.com/uc?id=1E5YwNKW2VOEayK9mWCS3Kpsxf-3z04ZE'

output = 'checkpoints/final.pth'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1ruJg4lqR_jgQPj-9K0PP-L2vJERYOxLP
To: /content/Self-Correction-Human-Parsing/checkpoints/final.pth
267MB [00:03, 81.7MB/s]


'checkpoints/final.pth'

In [ ]:
text = '''

#!/usr/bin/env python
# -*- encoding: utf-8 -*-

import os
import torch
import easydict
import numpy as np
from PIL import Image
from tqdm import tqdm

from torch.utils.data import DataLoader
import torchvision.transforms as transforms

import networks
from utils.transforms import transform_logits
from datasets.simple_extractor_dataset import SimpleFolderDataset

dataset_settings = {
    'lip': {
        'input_size': [473, 473],
        'num_classes': 20,
        'label': ['Background', 'Hat', 'Hair', 'Glove', 'Sunglasses', 'Upper-clothes', 'Dress', 'Coat',
                  'Socks', 'Pants', 'Jumpsuits', 'Scarf', 'Skirt', 'Face', 'Left-arm', 'Right-arm',
                  'Left-leg', 'Right-leg', 'Left-shoe', 'Right-shoe']
    },
    'atr': {
        'input_size': [512, 512],
        'num_classes': 18,
        'label': ['Background', 'Hat', 'Hair', 'Sunglasses', 'Upper-clothes', 'Skirt', 'Pants', 'Dress', 'Belt',
                  'Left-shoe', 'Right-shoe', 'Face', 'Left-leg', 'Right-leg', 'Left-arm', 'Right-arm', 'Bag', 'Scarf']
    },
    'pascal': {
        'input_size': [512, 512],
        'num_classes': 7,
        'label': ['Background', 'Head', 'Torso', 'Upper Arms', 'Lower Arms', 'Upper Legs', 'Lower Legs'],
    }
}


def get_arguments():
    """Parse all the arguments provided from the CLI.
    Returns:
      A list of parsed arguments.
    """
    args = easydict.EasyDict({
        "dataset": 'atr',
        "model_restore": 'checkpoints/final.pth',
        "gpu": '0',
        "input_dir": 'inputs',
        "output_dir": '/content/static/seg_outputs',
        "logits": False,
    })
    return args


def get_palette(num_cls):
    """ Returns the color map for visualizing the segmentation mask.
    Args:
        num_cls: Number of classes
    Returns:
        The color map
    """
    n = num_cls
    palette = [0] * (n * 3)
    for j in range(0, n):
        lab = j
        palette[j * 3 + 0] = 0
        palette[j * 3 + 1] = 0
        palette[j * 3 + 2] = 0
        i = 0
        while lab:
            palette[j * 3 + 0] |= (((lab >> 0) & 1) << (7 - i))
            palette[j * 3 + 1] |= (((lab >> 1) & 1) << (7 - i))
            palette[j * 3 + 2] |= (((lab >> 2) & 1) << (7 - i))
            i += 1
            lab >>= 3
    return palette


def main():
    args = get_arguments()

    gpus = [int(i) for i in args.gpu.split(',')]
    assert len(gpus) == 1
    if not args.gpu == 'None':
        os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu

    num_classes = dataset_settings[args.dataset]['num_classes']
    input_size = dataset_settings[args.dataset]['input_size']
    label = dataset_settings[args.dataset]['label']
    print("Evaluating total class number {} with {}".format(num_classes, label))

    model = networks.init_model('resnet101', num_classes=num_classes, pretrained=None)

    state_dict = torch.load(args.model_restore)['state_dict']
    from collections import OrderedDict
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[7:]  # remove `module.`
        new_state_dict[name] = v
    model.load_state_dict(new_state_dict)
    model.cuda()
    model.eval()

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.406, 0.456, 0.485], std=[0.225, 0.224, 0.229])
    ])
    dataset = SimpleFolderDataset(root=args.input_dir, input_size=input_size, transform=transform)
    dataloader = DataLoader(dataset)

    if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)

    palette = [255, 255, 255,
                0,0,0,
                0,0,0,
                0,0,0,
                0,0,0,
                0,0,0,
                0,0,0,
                0,0,0,
                0,0,0,
                0,0,0,
                0,0,0,
                0,0,0,
                0,0,0,
                0,0,0,
                0,0,0,
                0,0,0,
                0,0,0,
                0,0,0, ]
    with torch.no_grad():
        for idx, batch in enumerate(tqdm(dataloader)):
            image, meta = batch
            img_name = meta['name'][0]
            c = meta['center'].numpy()[0]
            s = meta['scale'].numpy()[0]
            w = meta['width'].numpy()[0]
            h = meta['height'].numpy()[0]

            output = model(image.cuda())
            upsample = torch.nn.Upsample(size=input_size, mode='bilinear', align_corners=True)
            upsample_output = upsample(output[0][-1][0].unsqueeze(0))
            upsample_output = upsample_output.squeeze()
            upsample_output = upsample_output.permute(1, 2, 0)  # CHW -> HWC

            logits_result = transform_logits(upsample_output.data.cpu().numpy(), c, s, w, h, input_size=input_size)
            parsing_result = np.argmax(logits_result, axis=2)
            parsing_result_path = os.path.join(args.output_dir, img_name[:-4] + '.png')
            output_img = Image.fromarray(np.asarray(parsing_result, dtype=np.uint8))
            output_img.putpalette(palette)
            output_img.save(parsing_result_path)
            if args.logits:
                logits_result_path = os.path.join(args.output_dir, img_name[:-4] + '.npy')
                np.save(logits_result_path, logits_result)
    return


if __name__ == '__main__':
    main()


'''
file = open("simple_extractor_edit_b.py","w")
file.write(text)
file.close()

In [ ]:
text = '''

#!/usr/bin/env python
# -*- encoding: utf-8 -*-

import sys
sys.path.append('/content/Self-Correction-Human-Parsing')

import os
import torch
import easydict
import numpy as np
from PIL import Image
from tqdm import tqdm

from torch.utils.data import DataLoader
import torchvision.transforms as transforms

import networks
from utils.transforms import transform_logits
from datasets.simple_extractor_dataset import SimpleFolderDataset

dataset_settings = {
    'lip': {
        'input_size': [473, 473],
        'num_classes': 20,
        'label': ['Background', 'Hat', 'Hair', 'Glove', 'Sunglasses', 'Upper-clothes', 'Dress', 'Coat',
                  'Socks', 'Pants', 'Jumpsuits', 'Scarf', 'Skirt', 'Face', 'Left-arm', 'Right-arm',
                  'Left-leg', 'Right-leg', 'Left-shoe', 'Right-shoe']
    },
    'atr': {
        'input_size': [512, 512],
        'num_classes': 18,
        'label': ['Background', 'Hat', 'Hair', 'Sunglasses', 'Upper-clothes', 'Skirt', 'Pants', 'Dress', 'Belt',
                  'Left-shoe', 'Right-shoe', 'Face', 'Left-leg', 'Right-leg', 'Left-arm', 'Right-arm', 'Bag', 'Scarf']
    },
    'pascal': {
        'input_size': [512, 512],
        'num_classes': 7,
        'label': ['Background', 'Head', 'Torso', 'Upper Arms', 'Lower Arms', 'Upper Legs', 'Lower Legs'],
    }
}


def get_arguments():
    """Parse all the arguments provided from the CLI.
    Returns:
      A list of parsed arguments.
    """
    args = easydict.EasyDict({
        "dataset": 'atr',
        "model_restore": 'checkpoints/final.pth',
        "gpu": '0',
        "input_dir": 'inputs',
        "output_dir": '/content/static/seg_outputs',
        "logits": False,
    })
    return args


def get_palette(num_cls):
    """ Returns the color map for visualizing the segmentation mask.
    Args:
        num_cls: Number of classes
    Returns:
        The color map
    """
    n = num_cls
    palette = [0] * (n * 3)
    for j in range(0, n):
        lab = j
        palette[j * 3 + 0] = 0
        palette[j * 3 + 1] = 0
        palette[j * 3 + 2] = 0
        i = 0
        while lab:
            palette[j * 3 + 0] |= (((lab >> 0) & 1) << (7 - i))
            palette[j * 3 + 1] |= (((lab >> 1) & 1) << (7 - i))
            palette[j * 3 + 2] |= (((lab >> 2) & 1) << (7 - i))
            i += 1
            lab >>= 3
    return palette


def main():
    args = get_arguments()

    gpus = [int(i) for i in args.gpu.split(',')]
    assert len(gpus) == 1
    if not args.gpu == 'None':
        os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu

    num_classes = dataset_settings[args.dataset]['num_classes']
    input_size = dataset_settings[args.dataset]['input_size']
    label = dataset_settings[args.dataset]['label']
    print("Evaluating total class number {} with {}".format(num_classes, label))

    model = networks.init_model('resnet101', num_classes=num_classes, pretrained=None)

    state_dict = torch.load(args.model_restore)['state_dict']
    from collections import OrderedDict
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k[7:]  # remove `module.`
        new_state_dict[name] = v
    model.load_state_dict(new_state_dict)
    model.cuda()
    model.eval()

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.406, 0.456, 0.485], std=[0.225, 0.224, 0.229])
    ])
    dataset = SimpleFolderDataset(root=args.input_dir, input_size=input_size, transform=transform)
    dataloader = DataLoader(dataset)

    if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)

    palette = [0, 0, 0,
                255, 255, 255,
                255, 255, 255,
                255, 255, 255,
                255, 255, 255,
                255, 255, 255,
                255, 255, 255,
                255, 255, 255,
                255, 255, 255,
                255, 255, 255,
                255, 255, 255,
                255, 255, 255,
                255, 255, 255,
                255, 255, 255,
                255, 255, 255,
                255, 255, 255,
                255, 255, 255,
                255, 255, 255, ]
    with torch.no_grad():
        for idx, batch in enumerate(tqdm(dataloader)):
            image, meta = batch
            img_name = meta['name'][0]
            c = meta['center'].numpy()[0]
            s = meta['scale'].numpy()[0]
            w = meta['width'].numpy()[0]
            h = meta['height'].numpy()[0]

            output = model(image.cuda())
            upsample = torch.nn.Upsample(size=input_size, mode='bilinear', align_corners=True)
            upsample_output = upsample(output[0][-1][0].unsqueeze(0))
            upsample_output = upsample_output.squeeze()
            upsample_output = upsample_output.permute(1, 2, 0)  # CHW -> HWC

            logits_result = transform_logits(upsample_output.data.cpu().numpy(), c, s, w, h, input_size=input_size)
            parsing_result = np.argmax(logits_result, axis=2)
            parsing_result_path = os.path.join(args.output_dir, img_name[:-4] + '.png')
            output_img = Image.fromarray(np.asarray(parsing_result, dtype=np.uint8))
            output_img.putpalette(palette)
            output_img.save(parsing_result_path)
            if args.logits:
                logits_result_path = os.path.join(args.output_dir, img_name[:-4] + '.npy')
                np.save(logits_result_path, logits_result)
    return


if __name__ == '__main__':
    main()

'''
file = open("simple_extractor_edit_w.py","w")
file.write(text)
file.close()

In [ ]:
def segmentation(file_name):
    ## segmentation
  src = '/content/static/anime_outputs/' + file_name
  dst = '/content/Self-Correction-Human-Parsing/inputs'
  shutil.copy(src, dst)

  os.chdir('/content/Self-Correction-Human-Parsing')

  try:
    subprocess.call(['python', '/content/Self-Correction-Human-Parsing/simple_extractor_edit_b.py']) # 사람 까맣게
  except:
    print('error')

  os.rename('/content/static/seg_outputs/' + file_name.split('.')[0] + '.png', '/content/static/seg_outputs/' + file_name.split('.')[0] + '_black.png')

  try:
    subprocess.call(['python', '/content/Self-Correction-Human-Parsing/simple_extractor_edit_w.py']) # 사람 까맣게
  except:
    print('error')

  os.rename('/content/static/seg_outputs/' + file_name.split('.')[0] + '.png', '/content/static/seg_outputs/' + file_name.split('.')[0] + '_white.png')

  os.chdir('/content')
  ## finish segmentation

  return "segmentation finished"

# 이미지 전처리

In [ ]:
white_img = np.full((256, 256), 255, dtype=np.uint8)
black_img = np.full((256, 256), 0, dtype=np.uint8)
color_img = np.full((256, 256, 3), (0, 255, 255), dtype=np.uint8)

In [ ]:
def pre_processing(black_mask, ani_img): # 이미지 경로 말고 이미지 이름 쓰는 거 임
  src = cv2.imread('/content/static/seg_outputs/' + black_mask, cv2.IMREAD_COLOR)        # 컬러 영상
  mask = cv2.imread('/content/static/seg_outputs/' + black_mask, cv2.IMREAD_GRAYSCALE)         # 그레이스케일 영상
  dst = cv2.imread('/content/static/anime_outputs/' + ani_img, cv2.IMREAD_COLOR)

  cv2.copyTo(src, mask, dst)
  cv2.imwrite('/content/static/seg_outputs/'+ ani_img.split('.')[0] + '_white_back.jpg', dst) # 이 이미지 보여주기

  return 'preprocessing OK', '/static/seg_outputs/'+ ani_img.split('.')[0] + '_white_back.jpg'

In [ ]:
def make_paddding(file_path):
  img = cv2.imread(file_path)
  a, b, c = img.shape

  differ = b - a
  differ_1, differ_2 = 0, 0

  if differ%2 == 0:
    differ_1, differ_2 = abs(differ/2), abs(differ/2)
  else:
    differ_1, differ_2 = abs(differ/2), abs(differ/2) + 1

  change_img = cv2.copyMakeBorder(img, 0, 0, int(differ_1), int(differ_2), cv2.BORDER_CONSTANT, value=[255, 255, 255])
  cv2.imwrite(file_path, change_img)

  return "make padding OK"

# 이미지 합성하기

In [ ]:
def splicing_back(idx, back_file_path, human_file):
  back = cv2.imread(back_file_path, cv2.IMREAD_COLOR)        # 컬러 영상
  back_resize = cv2.resize(back, dsize=(256*4, 256*2+60), interpolation=cv2.INTER_AREA)
  #cv2_imshow(back_resize)

  sp_idx = idx
  temp = cv2.imread('/content/static/seg_outputs/' + human_file.split('.')[0] + '_white.png', cv2.IMREAD_GRAYSCALE) 
  temp2 = cv2.imread('/content/Self-Correction-Human-Parsing/inputs/'+human_file, cv2.IMREAD_COLOR)


  print(back_resize.shape, " ", temp.shape, " ", temp2.shape)



  black_4 = cv2.hconcat([black_img, black_img, black_img, black_img])
  color_4 = cv2.hconcat([color_img, color_img, color_img, color_img])

  if sp_idx == 1 or sp_idx == 5:
    black_mask = cv2.hconcat([temp, black_img, black_img, black_img])
    color_mask = cv2.hconcat([temp2, color_img, color_img, color_img])
  elif sp_idx == 2 or sp_idx == 6:
    black_mask = cv2.hconcat([black_img, temp, black_img, black_img])
    color_mask = cv2.hconcat([color_img, temp2, color_img, color_img])
  elif sp_idx == 3 or sp_idx == 7:
    black_mask = cv2.hconcat([black_img, black_img, temp, black_img])
    color_mask = cv2.hconcat([color_img, color_img, temp2, color_img])
  elif sp_idx == 4 or sp_idx == 8:
    black_mask = cv2.hconcat([black_img, black_img, black_img, temp])
    color_mask = cv2.hconcat([color_img, color_img, color_img, temp2])

  if sp_idx > 4:
    back_mask = cv2.vconcat([black_4, black_mask])
    human_mask = cv2.vconcat([color_4, color_mask])
  else:
    back_mask = cv2.vconcat([black_mask, black_4])
    human_mask = cv2.vconcat([color_mask, color_4])

  #cv2_imshow(human_mask)

  black_edge = np.full((30, 1024), 0, dtype=np.uint8)
  color_edge = np.full((30, 1024, 3), (0, 255, 255), dtype=np.uint8)

  adflkajsdf = cv2.vconcat([back_mask, black_edge])
  back_mask = cv2.vconcat([black_edge, adflkajsdf])
  #cv2_imshow(back_mask)

  adkfalkdfd = cv2.vconcat([human_mask, color_edge])
  human_mask = cv2.vconcat([color_edge, adkfalkdfd])
  #cv2_imshow(human_mask)
  #print(back_mask.shape, back_resize.shape, human_mask.shape)

  cv2.copyTo(human_mask, back_mask, back_resize)
  cv2.imwrite('/content/static/splicing_outputs/splicing_result.jpg', back_resize)

  return "finish splicing"

# 웹페이지(HTML)

In [ ]:
%cd /content

/content


In [ ]:
text = '''
<html>

<head>
<title>Multi-Media Computing Website</title>
<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/css/bootstrap.min.css" />        
<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.1.0/jquery.min.js"></script>
</head>

    <body>
    <p><h1 align="center">Multi-Media Computing Website</h1></p>

<div class="container">
<div class="row">
    <h2>Select a file to upload</h2>
    <h5>합성하고자 하는 배경 사진과 인물 사진을 업로드 해 주세요.</h5>


     <form action = "/fileUpload" method = "POST" enctype = "multipart/form-data">
        <dl>
            <p>
                <input type="file" name="filename[]" multiple="multiple" class="form-control" autocomplete="off" required>
            </p>
        </dl>
        <p>
        <br><br>
        <div style = "text-align:center">
            <input type="submit" value="Submit" class="btn btn-info">
            </div?
        </p>
    </form>
</div>
</div>

        </form>
    </body>
</html>
'''
file = open("templates/upload.html","w")
file.write(text)
file.close()

In [ ]:
text = '''
<html>

<head>
<title>Multi-Media Computing Website</title>
<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/css/bootstrap.min.css" />        
<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.1.0/jquery.min.js"></script>
<script src="https://ajax.googleapis.com/ajax/libs/jquery/1.12.4/jquery.min.js"></script>
<script src="http://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/js/bootstrap.min.js"></script>
<script src="http://rawgithub.com/indrimuska/jquery-editable-select/master/dist/jquery-editable-select.min.js"></script>
<link href="http://rawgithub.com/indrimuska/jquery-editable-select/master/dist/jquery-editable-select.min.css" rel="stylesheet">
</head>


    <body>
    <div class="container">
        <h2 align="center"> Style Transfer Results - Animation ver</h2>
        <br><br>

  <div class="container; text-align:center">
<div style="float: left; padding: 10px; width: 50%;text-align:center">
Backgound Image
</div>

<div style="float: left; padding: 10px; width: 50%;text-align:center">
Human Image
</div>
  </div>

<br>


  <div class="container; text-align:center">
<div style="float: left; padding: 10px; width: 50%;text-align:center">
{% if True %}
	<img src="{{data1}}" alt="background_image"  width="400">
{% endif %}
</div>

<div style="float: left; padding: 10px; width: 50%;text-align:center">
{% if True %}
	<img src={{data2}} alt="human_image" width="256">
{% endif %}
</div>
</div>



</div>
<br><br><br>


<div class="container">
<div class="row">

    <form action = "/Seg_human" method = "POST">
      <input type = "hidden" name = "test2" value = {{file_name}} />        
       </select>
      </div>
      <div style = "text-align:center">
            <input type="submit" name='msg' value= "Segmentation" class="btn btn-info">
        </div>
        
    </form>

</div>
</div>

    </body>
</html>
'''

file = open("templates/showAnime.html","w")
file.write(text)
file.close()

In [ ]:
text = '''
<html>

<head>
<title>Multi-Media Computing Website</title>
<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/css/bootstrap.min.css" />        
<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.1.0/jquery.min.js"></script>
<script src="https://ajax.googleapis.com/ajax/libs/jquery/1.12.4/jquery.min.js"></script>
<script src="http://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/js/bootstrap.min.js"></script>
<script src="http://rawgithub.com/indrimuska/jquery-editable-select/master/dist/jquery-editable-select.min.js"></script>
<link href="http://rawgithub.com/indrimuska/jquery-editable-select/master/dist/jquery-editable-select.min.css" rel="stylesheet">
</head>


    <body>
    <div class="container">
        <h2 align="center"> Human Segmentation Results - Animation ver</h2>
        <br><br>

  <div class="container; text-align:center">
<div style="float: left; padding: 10px; width: 50%;text-align:center">
Backgound Image
</div>

<div style="float: left; padding: 10px; width: 50%;text-align:center">
Human Segmentation Image
</div>
  </div>

<br>


  <div class="container; text-align:center">
<div style="float: left; padding: 10px; width: 50%;text-align:center">
{% if True %}
	<img src="{{data1}}" alt="background_image"  width="400">
{% endif %}
</div>

<div style="float: left; padding: 10px; width: 50%;text-align:center">
{% if True %}
	<img src={{data2}} alt="human_image" width="256">
{% endif %}
</div>
</div>



</div>
<br><br><br>


<div class="container">
<div class="row">
    <h4 align="center">Select Position</h4>


    <form action = "/Select_pos" method = "POST">
      <div class="form-group" style="margin: auto; text-align:center; width: 650">
       <select name="position" id="position" class="form-control">
          <option value="">위치 선택</option>
          <option value="1">1</option>
          <option value="2">2</option>
          <option value="3">3</option>
          <option value="4">4</option>
          <option value="5">5</option>
          <option value="6">6</option>
          <option value="7">7</option>
          <option value="8">8</option>
       </select>
       <input type = "hidden" name = "test2" value = {{file_name}} /> 
      <Br><br><br>

            <input type="submit" value="Submit" class="btn btn-info">
        </div>
    </form>

</div>
</div>

    </body>
</html>
'''

file = open("templates/showSeg.html","w")
file.write(text)
file.close()

In [ ]:
text = '''
<html>

<head>
<title>Multi-Media Computing Website</title>
<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/css/bootstrap.min.css" />        
<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.1.0/jquery.min.js"></script>
<script src="https://ajax.googleapis.com/ajax/libs/jquery/1.12.4/jquery.min.js"></script>
<script src="http://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/js/bootstrap.min.js"></script>
<script src="http://rawgithub.com/indrimuska/jquery-editable-select/master/dist/jquery-editable-select.min.js"></script>
<link href="http://rawgithub.com/indrimuska/jquery-editable-select/master/dist/jquery-editable-select.min.css" rel="stylesheet">
</head>


    <body>
    <div class="container">
        <h2 align="center"> Our Result</h2>
        <br>

<br>


  <div class="container; text-align:center">
<div style="float: left; padding: 10px; width: 100%;text-align:center">
{% if True %}
	<img src="{{data1}}" alt="our_result_image"  width="700">
{% endif %}
</div>

</div>



</div>

    </body>
</html>
'''

file = open("templates/showResult.html","w")
file.write(text)
file.close()

# flask 코드

In [ ]:
%cd /content

/content


In [ ]:
from flask import Flask, render_template, request
from werkzeug.utils import secure_filename
import shutil
app = Flask(__name__)
run_with_ngrok(app)   


# 파일 업로드 처리
@app.route('/fileUpload', methods = ['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        files = request.files
        file_names = []
        print(type(files))
        print('-----------')
        for f in files.to_dict(flat=False)['filename[]']:
            f.save('static/inputs/' + secure_filename(f.filename))
            print(f.filename)
            print('파일 업로드 성공!(' ,secure_filename(f.filename) ,')')
            file_names.append(secure_filename(f.filename))
        pad_msg = make_paddding('static/inputs/' + file_names[1])
        print(pad_msg)
        return style_transfer(file_names)


# 업로드 HTML 렌더링
@app.route('/')
def render_file():
    return render_template('upload.html')
    


@app.route('/Select_pos', methods = ['GET', 'POST'])
def select_position():
  human_file = request.form.get('test2')
  select = request.form.get('position')
  splicing_msg = splicing_back(int(select), back_file_path, human_file)
  print(splicing_msg)
  return render_template('showResult.html', data1 = 'static/splicing_outputs/splicing_result.jpg')


@app.route('/Seg_human', methods = ['GET', 'POST'])
def seg_human():
  human_file = request.form.get('test2')

  seg_msg = segmentation(human_file)
  print(seg_msg)
  
  pro_msg, seg_file_path = pre_processing(human_file.split('.')[0] + '_black.png', human_file)
  print(pro_msg)

  return render_template('showSeg.html', data1 = back_file_path, data2 = seg_file_path, file_name = human_file)




def style_transfer(file_names):
  execfile('/content/animegan2-pytorch/anime_execute.py')
  print(file_names)
  global back_file_path
  back_file_path = 'static/anime_outputs/' + file_names[0]
  human_file_path = 'static/anime_outputs/' + file_names[1]
  msg = file_names[1]

  return render_template('showAnime.html', data1 = back_file_path, data2 = human_file_path, file_name = msg)
  #return "style_trnasfer OK"


if __name__ == '__main__':
    # 서버 실행
    app.run()

NameError: ignored